# ニュース記事のジャンル予測（9分類問題）

SetFitアルゴリズムを用いて、日本語Sentence-BERTモデルを分類タスク用に転移学習（ファインチューニング）します。

- 本実験の解説記事：[実際問題、Few-Shot学習手法SetFitはいつ使うとよいのか？](https://qiita.com/sonoisa/items/297fa2994a08c71d01c5)

SetFitアルゴリズム:
- 特徴: Sentence-BERTモデルを個別の分類タスクのよい特徴量になるように調整することで、few-shotでも高い精度を出せる。
- 論文: [Efficient Few-Shot Learning Without Prompts](https://arxiv.org/abs/2209.11055)
- リポジトリ: https://github.com/huggingface/setfit

![SetFitアルゴリズム](https://github.com/huggingface/setfit/raw/main/assets/setfit.png)
(図の出典: 上記SetFitのリポジトリ)

# ライブラリやデータの準備

## 依存ライブラリのインストール

In [1]:
!pip -q install setfit fugashi ipadic unidic_lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 47.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 697.9/697.9 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00


In [2]:
!pip list | grep -e setfit -e transformers -e torch -e fugashi -e ipadic

fugashi                               1.5.1
ipadic                                1.0.0
sentence-transformers                 5.1.0
setfit                                1.1.3
torch                                 2.8.0+cu126
torchao                               0.10.0
torchaudio                            2.8.0+cu126
torchdata                             0.11.0
torchsummary                          1.5.1
torchtune                             0.6.1
torchvision                           0.23.0+cu126
transformers                          4.56.1


## 各種ディレクトリ作成

* data: 学習用データセット格納用
* model: 学習済みモデル格納用

In [3]:
!mkdir -p /content/data /content/model

In [4]:
# 事前学習済みモデル
PRETRAINED_MODEL_NAME = "sonoisa/sentence-bert-base-ja-mean-tokens-v2"

# 転移学習済みモデルを保存する場所
MODEL_DIR = "/content/model"

## livedoor ニュースコーパスのダウンロード

In [5]:
!wget -O ldcc-20140209.tar.gz https://www.rondhuit.com/download/ldcc-20140209.tar.gz

--2025-09-15 06:32:13--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  2.71MB/s    in 3.1s    

2025-09-15 06:32:16 (2.71 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



## livedoorニュースコーパスの形式変換

livedoorニュースコーパスを次の形式のJSONファイルに変換します。

* "label": ジャンルID（0〜8）
* "text": タイトル + 本文

JSONファイルは/content/dataに格納されます。

## 文字列の正規化の定義

表記揺れを減らします。今回は[neologdの正規化処理](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja)を一部改変したものを利用します。
処理の詳細はリンク先を参照してください。

In [6]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

## 情報抽出

ニュース記事のタイトルと本文とジャンル（9分類）の情報を抽出します。

In [7]:
import tarfile
import re

target_genres = ["dokujo-tsushin",
                 "it-life-hack",
                 "kaden-channel",
                 "livedoor-homme",
                 "movie-enter",
                 "peachy",
                 "smax",
                 "sports-watch",
                 "topic-news"]

def remove_brackets(text):
    text = re.sub(r"(^【[^】]*】)|(【[^】]*】$)", "", text)
    return text

def normalize_text(text):
    assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

def read_title_body(file):
    next(file)
    next(file)
    title = next(file).decode("utf-8").strip()
    title = normalize_text(remove_brackets(title))
    body = normalize_text(" ".join([line.decode("utf-8").strip() for line in file.readlines()]))
    return title, body

genre_files_list = [[] for genre in target_genres]

all_data = []

with tarfile.open("ldcc-20140209.tar.gz") as archive_file:
    for archive_item in archive_file:
        for i, genre in enumerate(target_genres):
            if genre in archive_item.name and archive_item.name.endswith(".txt"):
                genre_files_list[i].append(archive_item.name)

    for i, genre_files in enumerate(genre_files_list):
        for name in genre_files:
            file = archive_file.extractfile(name)
            title, body = read_title_body(file)
            title = normalize_text(title)
            body = normalize_text(body)

            if len(title) > 0 and len(body) > 0:
                all_data.append({
                    "title": title,
                    "body": body,
                    "genre_id": i
                    })

## データ分割

データセットを70% : 15%: 15% の比率でtrain/dev/testに分割します。

* trainデータ: 学習に利用するデータ
* devデータ: 学習中の精度評価等に利用するデータ
* testデータ: 学習結果のモデルの精度評価に利用するデータ

In [8]:
import random
from tqdm import tqdm
import json

random.seed(1234)
random.shuffle(all_data)

def to_line(data):
    title = data["title"]
    body = data["body"]
    genre_id = data["genre_id"]

    assert len(title) > 0 and len(body) > 0
    return json.dumps({
        "label": genre_id,
        "text": f"{title} {body}"
    }, ensure_ascii=False) + "\n"

data_size = len(all_data)
train_ratio, dev_ratio, test_ratio = 0.7, 0.15, 0.15

with open(f"data/train_full.json", "w", encoding="utf-8") as f_train, \
    open(f"data/dev.json", "w", encoding="utf-8") as f_dev, \
    open(f"data/test.json", "w", encoding="utf-8") as f_test:

    for i, data in tqdm(enumerate(all_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        elif i < (train_ratio + dev_ratio) * data_size:
            f_dev.write(line)
        else:
            f_test.write(line)

7334it [00:00, 49461.15it/s]


作成されたデータを確認します。

形式: { "label": 分類ラベル, "text": 分類する文章 }

In [9]:
!head -3 data/test.json

{"label": 6, "text": "nttドコモ、ジョジョの奇妙な冒険25周年スマホ「jojo l-06d」を発表!荒木飛呂彦氏監修コンテンツが満載の全部入り[optimus_report] オラララオラオラオラオラオラオラオラオラオラオラ!nttドコモは16日、今夏に発売する予定の新モデルや新しく開始するサービスなどを発表する「2012年夏モデル新商品・新サービス発表会」を開催し、人気マンガ「ジョジョの奇妙な冒険」の連載25周年を記念した限定モデル「jojo l-06d」(lgエレクトロニクス製)を発表しています。発売時期は2012年8月を予定しています。jojo l-06dは限定1万5,000台の限定モデルで、5インチサイズの大型ディスプレイを搭載したxi対応androidスマートフォン「optimus vu l-06d」をベースに、原作者・荒木飛呂彦氏が監修したコラボレーションモデルです。荒木氏は監修のほか、jojo l-06dのためだけの書き下ろしイラスト&サインが入っており、ジョジョ好きにはたまらないコンテンツが満載です。コンテンツには、荒木氏が書き下ろした壁紙を含むジョジョの人気イラストの壁紙やライブ壁紙を多数プリインストール。さらに、6種類のきせかえテーマと組み合わせることで、自分だけのお気に入りのホーム画面を設定可能になっています。また、ジョジョ第3部に登場するカーレースゲーム「f-mega」もプリインストールされており、花京院とダービー弟の名勝負を体験できます。さらに、お気に入りのスタンドと合成できるカメラアプリや、トリッシュの電卓、ウェザー・リポートウィジェット、イギーのマチキャラというように作品中に登場するキャラクターによる各種機能、「ジョジョ」の名台詞を織り交ぜたオリジナルの予測変換辞書、オリジナルデコメ絵文字、デコメテンプレートなども搭載。この他、画面サイズが4:3でほぼ文庫サイズのl-06dの端末機能を活かして、特別編集のカラー版コミック第1巻〜12巻も内蔵されています。機能的にも、optimus vu l-06dと同等で、高速データ通信規格lteによるサービス「xi(クロッシィ)」による下り最大75mbpsおよび上り最大25mbpsの高速データ通信や1.5ghzデュアルコアcpu、5インチxga(769×1024ドット)ips液

# SetFit用の訓練データ作成

各クラスのデータ数がデータ数が8個になるようにする（層化抽出）。

In [10]:
import json
from datasets import load_dataset

class_labels = list(range(9))  # 分類ラベルのリスト
samples_per_class = 8  # クラスあたりのデータ数

train_dataset = load_dataset("json", data_files="/content/data/train_full.json")["train"]
train_dataset = train_dataset.shuffle(seed=5678)

with open("/content/data/train_fewshot.json", "w", encoding="utf-8") as f_out:
    for class_label in class_labels:
        class_data = train_dataset.filter(lambda x: x["label"] == class_label).select(range(samples_per_class))

        assert len(class_data) == samples_per_class

        for data in class_data:
            f_out.write(json.dumps(data, ensure_ascii=False))
            f_out.write("\n")


Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/5134 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5134 [00:00<?, ? examples/s]

In [11]:
# 全データを利用する場合
# train_dataset = load_dataset("json", data_files="/content/data/train_full.json")["train"].shuffle(seed=42)

# サンプリングされたデータを利用する場合
train_dataset = load_dataset("json", data_files="/content/data/train_fewshot.json")["train"].shuffle(seed=42)
eval_dataset = load_dataset("json", data_files="/content/data/test.json")["train"]

print(f"train_dataset: {len(train_dataset)}")
for sample in train_dataset:
    # print(sample)
    pass

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

train_dataset: 72


# 訓練と精度評価

In [12]:
import os
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, Trainer, TrainingArguments

# W&BのAPIキー入力が求められるのを避ける
os.environ["WANDB_DISABLED"] = "true"

model = SetFitModel.from_pretrained(PRETRAINED_MODEL_NAME)

args = TrainingArguments(
    batch_size=16,
    num_iterations=20,
    num_epochs=1,
    body_learning_rate=2e-5,
    head_learning_rate=2e-5,
    seed=42,
    loss=CosineSimilarityLoss,
    report_to="tensorboard",
)

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=args,
)
trainer.train()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: d

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 2880
  Batch size = 16
  Num epochs = 1


Step,Training Loss
1,0.329100
50,0.147700
100,0.061000
150,0.027100


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:451: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  opt_res = optimize.minimize(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [19]:
trainer.model.save_pretrained(MODEL_DIR)

In [17]:
metrics = trainer.evaluate()
print(metrics)

***** Running evaluation *****


{'accuracy': 0.6327272727272727}


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
